In [1]:
import os, time, datetime
import sys
import torch
import torch.optim as optim
from tqdm import tqdm, trange
from torchvision import transforms, datasets
from torchvision.utils import make_grid
import argparse
from argparse import Namespace

from nice import NICE
from dircluster import sample_mu_lam, SSE, LLH
from utils import savefig_clusters, mvnlogpdf, plt2img
from AE import AE

import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
from numpy import log, exp, pi
from scipy.stats import wishart, gamma
from scipy.stats import multivariate_normal as normal
from numpy.linalg import inv, det
from matplotlib.patches import Ellipse
from scipy.special import loggamma
import tensorflow as tf
from scipy.stats import wishart, gamma, entropy
# from plot_cf import pretty_plot_confusion_matrix
import pandas as pd
from numpy import random
from mymetrics import v_measure_score, adjusted_rand_score, pair_confusion_matrix, normalized_mutual_info_score, adjusted_mutual_info_score
# matplotlib.use('Agg')
from ae_util import *
from utils import *
from sklearn.datasets import make_blobs
from sklearn import datasets
import warnings 
warnings.filterwarnings('ignore')

TRAIN_BATCH_SIZE = 128
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # using specific GPU

2022-02-12 10:47:48.636425: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
seed=123456
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
def get_iter_num(n_epoch, n_samples, cfg_list):
    '''
    How many steps for iteration in the n-th epoch
    num_iter is configed to decay for fast training 
    '''
    ratio = cfg_list[-1] if n_epoch >= len(cfg_list) else cfg_list[n_epoch]
    return int(np.ceil(n_samples * ratio))


def cluster2label(K, samples_k, y):
    map_dict = {}
    y_pred = samples_k.copy()
    for k_idx in range(K):  # analysis each cluster
        cluster_sid = (samples_k == k_idx)
        label, counts = np.unique(y[cluster_sid], return_counts=True)
        major_class_idx = np.argmax(counts)

        map_dict[k_idx] = label[major_class_idx]
        y_pred[cluster_sid] = label[major_class_idx]
    return y_pred, map_dict


def pair_f1_score(ys, ks):
    cf = pair_confusion_matrix(ys, ks)
    # TN, FP
    # FN, TP
    return cf[1, 1] / float(cf[1, 1] + 0.5 * (cf[0, 1] + cf[1, 0]))


def get_count_matrix(K, cluster_label, nC, label):
    # count labels in each cluster
    np.unique(cluster_label)
    cluster_counts_list = []
    for k in range(K):
        c_sample_id = (cluster_label == k)
        y, ny = np.unique(label[c_sample_id], return_counts=True)
        counts_arr = np.zeros(nC)
        counts_arr[y] = ny
        cluster_counts_list.append(counts_arr)
    count_matrix = np.stack(cluster_counts_list, axis=0)
    return count_matrix


def cluster_acc_fixed(Y_pred, Y):  # from paper VaDE code
    from scipy.optimize import linear_sum_assignment as linear_assignment
    assert Y_pred.size == Y.size
    D = max(Y_pred.max(), Y.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    a, b = linear_assignment(w.max() - w)
    ind = [(a[i], b[i]) for i in range(a.shape[0])]
    return sum([w[i, j] for i, j in ind]) * 1.0 / Y_pred.size, ind


def eval_cluster(samples_k, ys, return_extra=False, alg='default', ds_name='default'):
    cids, nK = np.unique(samples_k, return_counts=True)
    k_indices = np.argsort(-nK)  # desc sort by size
    k_major, k_entropy = [], []
    confusion_matrix = np.zeros(shape=(10, 10), dtype=float)
    correct = 0
    for k_idx in k_indices:  # analysis each cluster
        cluster_sid = (samples_k == cids[k_idx])
        cluster_size = nK[k_idx]
        label, counts = np.unique(ys[cluster_sid], return_counts=True)
        k_entropy.append(entropy(counts / cluster_size))

        major_class_idx = np.argmax(counts)
        k_major.append(label[major_class_idx])
        confusion_matrix[label, label[major_class_idx]] += counts
        correct += counts[major_class_idx]
    acc = correct / np.sum(nK)
    weighted_entropy = np.sum(nK[k_indices]/np.sum(nK) * k_entropy)

    # k-irrevlent measure
    v_score, adj_rand = v_measure_score(ys, samples_k), adjusted_rand_score(ys, samples_k),
    pair_f1 = pair_f1_score(ys, samples_k)
    if not return_extra: 
        return acc, v_score, adj_rand, pair_f1, weighted_entropy, confusion_matrix
    else:
        nmi = normalized_mutual_info_score(ys, samples_k) 
        return pair_f1, v_score, adj_rand, nmi, acc, len(cids), weighted_entropy, confusion_matrix


def dirichlet_clustering(global_epoch, tb_writter, dir_params, samples, ys, n_iter_samples, args, log_step=0):
    hp = dir_params.hyper
    _mu0, _ka0, logalpha, _a0, _b0 = hp.mu0, hp.ka0, hp.logalpha, hp.a0, hp.b0
    K, lam_K, mu_K, nK, ks = dir_params.K, dir_params.lam_K, dir_params.mu_K, dir_params.n_K, dir_params.samples_k

    # begin iter
    monitor_freq = 100
    with tb_writter.as_default():
        for iter_idx in range(n_iter_samples):
            ## report result
            global_log_step = iter_idx+log_step
            sample_idx = global_log_step % len(ks)  # mod by n_sample_load

            # others
            if (global_log_step + 1) % monitor_freq == 0:
                # report cluster information
                tf.summary.scalar('dmm/K', K, step=iter_idx+log_step)
                arr = np.asarray(nK)/sum(nK)
                tf.summary.histogram('cluster size ratio', arr, step=global_log_step)
                mu_norm = np.linalg.norm(np.asarray(mu_K), axis=1)
                tf.summary.histogram('cluster mu norm', mu_norm, step=global_log_step)
                tf.summary.histogram('cluster lambda', np.asarray(lam_K), step=global_log_step)

                # report metrics
#                 if (global_log_step + 1) % (monitor_freq*10) == 0:
                error = SSE(samples, K, ks, nK, mu_K, lam_K)
                pair_f1, v_score, adj_rand, nmi, acc, _, w_ent, _ = eval_cluster(ks, ys,return_extra=True, ds_name=args.dataset)
                tf.summary.scalar('dmm/sse', error, step=global_log_step)
                tf.summary.scalar('dmm/acc', acc, step=global_log_step)
                tf.summary.scalar('dmm/v_score', v_score, step=global_log_step)
                tf.summary.scalar('dmm/nmi', nmi, step=global_log_step)
                tf.summary.scalar('dmm/adj_rand', adj_rand, step=global_log_step)
                tf.summary.scalar('dmm/pair_f1', pair_f1, step=global_log_step)
                tf.summary.scalar('dmm/w_entropy', w_ent, step=global_log_step)
                tb_writter.flush()
                
            xi = samples[sample_idx]
            old_k = ks[sample_idx]
            if old_k != -1: 
                nK[old_k] -= 1
                if nK[old_k] == 0: 
                    idx = (ks == K - 1)
                    ks[idx] = old_k
                    nK[old_k], lam_K[old_k], mu_K[old_k], = nK[K - 1], lam_K[K - 1], mu_K[K - 1]
                    nK, lam_K, mu_K = nK[:-1], lam_K[:-1], mu_K[:-1]
                    K -= 1

            p_lst = []  # p_lst[k] is the probability that xi in C_k, p_lst[-1] means xi forms a new cluster
            Kn = K 
            if K == 0:  # if no cluster, just build a new one
                chosen_k = 0
            else:
                # existing cluster assignment prob
                Klst = np.random.choice(K, size=Kn, replace=False, p=np.asarray(nK) / np.sum(np.asarray(nK)))
                for k in Klst:
                    pk = log(nK[k]) + mvnlogpdf(xi, mu_K[k], 1 / lam_K[k])
                    p_lst.append(pk)

                # new cluster assignment pob.
                _kan = _ka0 + 1
                _an = _a0 + 0.5 * args.dim
                _bn = _b0 + _ka0 * np.linalg.norm(xi - _mu0) ** 2 / (2 * (_ka0 + 1))
                logpk = log(logalpha) + \
                        loggamma(_an) - loggamma(_a0) + \
                        _a0 * log(_b0) - _an * log(_bn) + \
                        0.5 * (log(_ka0) - log(_kan)) - args.dim / 2 * log(2 * pi)
                p_lst.append(logpk)

                # sampling according to the assignment prob.
                maxpk = max(p_lst)
                p_lst = [exp(v - maxpk) for v in p_lst]  
                chosen_k = np.random.choice(list(range(Kn + 1)),
                                            p=p_lst / sum(p_lst))  # sample, now xi belongs cluster chosen_k

            # 3.1.2 Sampling Cluster mean and lambda
            if chosen_k == Kn:  # assigned to new cluster
                nK.append(1)
                ks[sample_idx] = K
                lam_k, mu_k = sample_mu_lam(samples, nK, ks, chosen_k, _mu0, _ka0, _a0, _b0)
                mu_K.append(mu_k)
                lam_K.append(lam_k)
                K += 1
            else:  # assigned to existing cluster
                chosen_k = Klst[chosen_k]
                nK[chosen_k] += 1
                ks[sample_idx] = chosen_k
                
            if sample_idx % args.dmm_rebuild_freq == 0 or sample_idx == args.n_sample_load:
                for k in range(K):
                    lam_K[k], mu_K[k] = sample_mu_lam(samples, nK, ks, k, _mu0, _ka0, _a0, _b0)

        # the last iter
        pass
        tf.summary.scalar('dmm_epoch/sse', error, step=global_epoch)
        tf.summary.scalar('dmm_epoch/acc', acc, step=global_epoch)
        tf.summary.scalar('dmm_epoch/v_score', v_score, step=global_epoch)
        tf.summary.scalar('dmm_epoch/adj_rand', adj_rand, step=global_epoch)
        tf.summary.scalar('dmm_epoch/pair_f1', pair_f1, step=global_epoch)
        tf.summary.scalar('dmm_epoch/nmi', nmi, step=global_epoch)
        tf.summary.scalar('dmm_epoch/w_entropy', w_ent, step=global_epoch)
    # update params
    dir_params.K, dir_params.lam_K, dir_params.mu_K, dir_params.n_K, dir_params.samples_k = K, lam_K, mu_K, nK, ks
    return dir_params

def train_flow(global_epoch, tb_writter, model_nice, opt, ds_inf_iter, n_iter, dir_params, args,
               log_step=0, prev_z_repr=None):
    if n_iter == 0:
        print('flow_opt=0, train ignored')
        return
    model_nice = model_nice.to(args.device)
    epoch_n_batch = args.n_sample_load // TRAIN_BATCH_SIZE
    epoch_loss, epoch = 0.0, 0
    K, ks, ns, mu_K, lam_K = dir_params.K, dir_params.samples_k, dir_params.n_K, dir_params.mu_K, dir_params.lam_K

    model_nice.to(args.device)
    with tb_writter.as_default():
        for n_batch in range(n_iter):
            (_, x, _, idx) = next(ds_inf_iter)
            x = x.to(args.device)

            model_nice.train()
            opt.zero_grad()
            x = x.to(args.device)
            _, likelihood = model_nice(x, K, ks[idx.numpy()], ns, np.asarray(mu_K), np.asarray(lam_K))
            if isinstance(likelihood, int):
                continue
            loss = -torch.mean(likelihood)  # NLL
            
            loss.backward() 
            opt.step() 


            tf.summary.scalar('flow/batch_nll_loss', loss.detach().cpu().numpy(), step=n_batch + global_epoch * n_iter)
            if (n_batch + 1) % epoch_n_batch == 0:
                tb_writter.flush()
                tf.summary.scalar('flow/epoch_nll_loss', epoch_loss / epoch_n_batch,
                                  step=int(epoch + global_epoch * (n_iter / epoch_n_batch)))
                epoch_loss *= 0.0
                epoch += 1
            else:
                epoch_loss += loss.detach().cpu().numpy() 


def init_dir_params(args):
    dir_params = Namespace(
        hyper=Namespace(
            a0=args.a0 * args.dim, b0=args.b0 * args.dim,
            mu0=np.zeros(args.dim), ka0=args.kappa0,
            logalpha=args.logalpha
        ),
        K=0, n_K=[], lam_K=[], mu_K=[], samples_k=np.ones(args.n_sample_load, dtype=int) * -1
        # K: total num of clusters. lam_K, mu_K: sampled mu/lambda cluster. n_K: size of clusters
        # samples_k: cluster index for each sample, -1 means unassigned
    )
    return dir_params


def init_flow_model(args):
    model_nice = NICE(data_dim=args.dim, num_coupling_layers=args.nice_nlayers,
                      num_hidden_units=args.nice_units, device_name=args.device)
#     opt = optim.SGD(model_nice.parameters(), args.lr)
    opt = optim.Adam(model_nice.parameters(), args.lr)
    return model_nice, opt


def load_ae_dataset(ds_name, n_sample_load, aex_file, idx_load=None):
    print('start loading ae dataset')
    if ds_name == 'mnist':
        ds_raw = load_dataset(ds_name, './data', split='train')
        if idx_load is None:
            ds_ae_repr = torch.load(aex_file)[:n_sample_load]
        else:
            ds_ae_repr = torch.load(aex_file)[idx_load]
        ds_ae = AEDataset(ds_raw, ds_ae_repr, n_sample_load, idx_load=idx_load)
    elif 'VaDE_' in ds_name:
        X, Y, Z = np.load(f'../VaDE/{ds_name}_full_X.npz')['arr_0'], \
                  np.load(f'../VaDE/{ds_name}_full_Y.npz')['arr_0'].reshape(-1, ), \
                  np.load(f'../VaDE/{ds_name}_full_Z.npz')['arr_0']
        # X = (X - X.mean()) / X.std()  # todo critical norm
        if idx_load is None:
            ds_ae_repr = torch.from_numpy(Z)[:n_sample_load]
        else:
            ds_ae_repr = torch.from_numpy(Z)[idx_load]
        ds_ae = OtherAEDataset(X, Y, Z, n_sample_load, idx_load=idx_load)
    else:
        raise ValueError(f'dataset name {ds_name} not supported')
    return ds_ae_repr, ds_ae

In [4]:
class TmpArgs:
    def __init__(self):
        self.dataset = 'VaDE_stl'
        self.dim = 10
        self.noise = 0.0 
        self.n_sample_full = self.n_sample_load = 13000
        self.epoch = 5   #30
        self.dir_epoch = 3  #10
        self.iter_dmm = self.n_sample_load*3
        self.dmm_rebuild_freq = 50 # self.n_sample_load // 50
        self.iter_nice = self.n_sample_load // 5
        self.save_freq = 1
        self.lr = 0.000001
        self.logalpha = 1e-10
        self.kappa0 = 0.005
        self.a0 = 1000
        self.b0 = 100
        self.nice_nlayers = 6
        self.nice_units = 512
        self.exp_name = 'stl'
        self.pretrainpath = None
        self.aex_file = None
        self.device = 'cuda'
        self.log_dir = './exp_out/stl'
        
args = TmpArgs()    #parse_args(manual_args)

## main

In [5]:
time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_id = 'dir_%s_%s_N%d_E%d_kappa0_%s_a0_%s_b0_%s_alpha_%s_%s_p%s' % (
        args.exp_name, args.dataset, args.n_sample_load, args.epoch, args.kappa0, args.a0, args.b0, args.logalpha, time_str, os.getpid())
task_dir = os.path.join(args.log_dir, exp_id)
if not os.path.exists(task_dir):
    os.makedirs(task_dir)

model_nice, opt = init_flow_model(args)
dir_params = init_dir_params(args)
ds_ae_repr, ds_ae = load_ae_dataset(args.dataset, args.n_sample_load, args.aex_file)
samples_flow_z = transform_z(model_nice, ds_ae, args.n_sample_load)
dl_ae = InfiniteDataLoader(dataset=ds_ae, batch_size=TRAIN_BATCH_SIZE, shuffle=True, pin_memory=True)
iter_ae_inf = iter(dl_ae)
print(f'data loaded, cls dist: N={args.n_sample_load}'
      f'{np.stack(np.unique(ds_ae.targets, return_counts=True))}')

tb_logger = tf.summary.create_file_writer(task_dir)

global_step_dmm, global_step_flow = 0, 0

# samples_flow_z = transform_z(model_nice, ds_ae, args.n_sample_load)
for n_epoch in range(args.dir_epoch):
    print(f'{os.getpid()}-DDPM training epoch {n_epoch}')
    n_iter_clst = args.iter_dmm
    dir_params = dirichlet_clustering(n_epoch, tb_logger, dir_params, samples_flow_z, ds_ae.targets, n_iter_clst, args, log_step=global_step_dmm)
    global_step_dmm += n_iter_clst+1

start loading ae dataset
transfrom consumed 22.2s
data loaded, cls dist: N=13000[[   0    1    2    3    4    5    6    7    8    9]
 [1300 1300 1300 1300 1300 1300 1300 1300 1300 1300]]
14565-DDPM training epoch 0


2022-02-12 10:48:14.833461: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-12 10:48:14.834067: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-12 10:48:14.834815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:84:00.0 name: Tesla M60 computeCapability: 5.2
coreClock: 1.1775GHz coreCount: 16 deviceMemorySize: 7.94GiB deviceMemoryBandwidth: 149.31GiB/s
2022-02-12 10:48:14.834835: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-12 10:48:14.834872: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-12 10:48:14.834892: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-02-12 

14565-DDPM training epoch 1
14565-DDPM training epoch 2


In [6]:
exp_id = 'full_%s_%s_N%d_E%d_kappa0_%s_a0_%s_b0_%s_alpha_%s_%s_p%s' % (
        args.exp_name, args.dataset, args.n_sample_load, args.epoch, args.kappa0, args.a0, args.b0, args.logalpha, time_str, os.getpid())
task_dir = os.path.join(args.log_dir, exp_id)
if not os.path.exists(task_dir):
    os.makedirs(task_dir)

model_nice, opt = init_flow_model(args)
# ds_ae_repr, ds_ae = load_ae_dataset(args.dataset, args.n_sample_load, args.aex_file)
# samples_flow_z = transform_z(model_nice, ds_ae, args.n_sample_load)
# 4.alternated training
tb_logger = tf.summary.create_file_writer(task_dir)

global_step_dmm, global_step_flow = 0, 0

for n_epoch in range(args.epoch):
    print(f'{os.getpid()}-DDPM training epoch {n_epoch}') 
    samples_flow_z = transform_z(model_nice, ds_ae, args.n_sample_load) 
    n_iter_clst = args.iter_dmm
    dir_params = dirichlet_clustering(n_epoch, tb_logger, dir_params, samples_flow_z, ds_ae.targets, n_iter_clst, args, log_step=global_step_dmm)
    global_step_dmm += n_iter_clst+1
    
    n_iter_opt = args.iter_nice
    train_flow(n_epoch, tb_logger, model_nice, opt, iter_ae_inf, n_iter_opt, dir_params, args,
               log_step=global_step_flow, prev_z_repr=samples_flow_z)
    global_step_flow += n_iter_opt+1

    # save params
    if (n_epoch+1) % args.save_freq == 0 or n_epoch == args.epoch-1:
        # save flow model
        torch.save(model_nice.state_dict(), os.path.join(task_dir, f'flow_E{n_epoch}.pt'))

        # save dpm params
        np.savez(os.path.join(task_dir, f'dpm_E{n_epoch}'), K=dir_params.K, ks=dir_params.samples_k, ns=dir_params.n_K,
                 mu_K=np.asarray(dir_params.mu_K), lam_K=np.asarray(dir_params.lam_K))

14565-DDPM training epoch 0
transfrom consumed 22.9s
14565-DDPM training epoch 1
transfrom consumed 23.1s
14565-DDPM training epoch 2
transfrom consumed 23.0s
14565-DDPM training epoch 3
transfrom consumed 30.2s
14565-DDPM training epoch 4
transfrom consumed 23.1s
